### Team Members
* Sean Tran - 101449600
* Mohammed Mujtaba Rabbani  - 101387404

### Table of Content
* Abstract
* GPT as Text Generator
* Experiment with hyperparameters
* BIAS in GPT
* Few-Shot Learning
1. Q/A
2. Context-based Answering
3. Abstract Summary

* Prompt Engineering
1. Language Translation
2. Write a friendly email
3. Write an Instagram Caption
4. Write a Comical Haiku
5. Instruction to build a rocket
6. Give Life advice
7. Give Exercise Advice
8. Give a 20-minute Exercise instruction
9. Write an AI model
10. Give Life-Coach advice
11. Help with writing essay
12. Research on environmental impact
* Summary
* Conclusion
* References


### Abstract
* For the second part of our math project, we look at GPT and its capabilities, focusing more on the application side rather than the training side.
* We explore various applications of the large GPT2 available from Hugging Face.
* We explore Prompt Engineering using the latest commercially available GPT.
* Some part can be lengthy information regarding tensors, skip ahead if not interested

In [1]:
!pip install transformers

from transformers import pipeline, set_seed, GPT2Tokenizer, GPT2LMHeadModel
from torch import tensor, numel  # numel to count up the number of parameters

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 8.8 MB/s eta 0:00:00


In [2]:
!pip install bertviz
from bertviz import model_view

## GPT as Text Generator
* Play "Finish The Lyrics"
* "Just stay on the track and never look back!"

In [21]:
set_seed(42)
generator = pipeline('text-generation', model='gpt2')

In [64]:
At_the_beginning = "Just stay on the track and never look back!"

generator(At_the_beginning, max_length=30, num_return_sequences=3) # keep generate text til limit is reached, randomness matter to vary output options

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Just stay on the track and never look back! I'm so happy that you didn't have to buy a whole new car! Thanks again.\n"},
 {'generated_text': 'Just stay on the track and never look back!\n\nAll our products are made to order. Our products are hand made in the USA, so'},
 {'generated_text': "Just stay on the track and never look back! You all have the right to tell us what you like, what you don't like (I will"}]

### Inner working of GPT, SKIP AHEAD IF NOT INTERESTED

In [65]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer.convert_ids_to_tokens(tokenizer.encode(At_the_beginning))

['Just',
 'Ġstay',
 'Ġon',
 'Ġthe',
 'Ġtrack',
 'Ġand',
 'Ġnever',
 'Ġlook',
 'Ġback',
 '!']

In [66]:
encoded = tokenizer.encode(At_the_beginning, return_tensors='pt')
encoded

tensor([[5703, 2652,  319,  262, 2610,  290, 1239,  804,  736,    0]])

In [67]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
# model

In [69]:
model.transformer.wte(encoded).shape

torch.Size([1, 10, 768])

In [71]:
max_length = model.transformer.wte(encoded).shape[1]
max_length

10

In [76]:
model.transformer.wpe(torch.arange(max_length).reshape(1,max_length)).shape   # how the position vector is transformed

torch.Size([1, 10, 768])

In [77]:
# same as unsqueeze
model.transformer.wpe(torch.arange(max_length).unsqueeze(dim=0)).shape

torch.Size([1, 10, 768])

In [80]:
initial_input = model.transformer.wte(encoded) + model.transformer.wpe(torch.arange(max_length).reshape(1,max_length))
initial_input.shape

torch.Size([1, 10, 768])

In [81]:
initial_input = model.transformer.drop(initial_input)
initial_input

tensor([[[ 0.0114, -0.2601,  0.0581,  ...,  0.0329, -0.0573,  0.0072],
         [ 0.1354, -0.0438,  0.0196,  ..., -0.1039, -0.1521,  0.3092],
         [-0.0454, -0.0329,  0.0899,  ..., -0.1099, -0.1276,  0.0272],
         ...,
         [ 0.0849, -0.0269,  0.3206,  ..., -0.1601,  0.1574, -0.0744],
         [ 0.0025,  0.0542,  0.1458,  ..., -0.0118, -0.1043, -0.1622],
         [-0.1052, -0.0371,  0.1509,  ..., -0.1367,  0.0191,  0.0398]]],
       grad_fn=<AddBackward0>)

In [82]:
model.lm_head

Linear(in_features=768, out_features=50257, bias=False)

In [83]:
for module in model.transformer.h:              # running our input thru each layer
  initial_input = module(initial_input)[0]

initial_input = model.transformer.ln_f(initial_input)
initial_input

tensor([[[-0.1124,  0.0987, -0.2257,  ..., -0.2548, -0.0302, -0.0129],
         [ 0.7490, -0.3286, -0.1462,  ...,  0.0615, -0.0844,  0.3857],
         [ 0.4196,  0.3504, -0.1835,  ..., -0.2092,  0.6564,  0.0437],
         ...,
         [ 0.7356,  0.1710, -1.2289,  ..., -0.0389,  0.5717,  0.0603],
         [ 0.5751, -0.4045, -2.5969,  ..., -0.0122,  0.3050,  0.1939],
         [ 0.1517, -0.4045, -0.1892,  ..., -0.2258, -0.1274, -0.2186]]],
       grad_fn=<NativeLayerNormBackward0>)

In [84]:
model(encoded, output_hidden_states=True).hidden_states[-1]

tensor([[[-0.1124,  0.0987, -0.2257,  ..., -0.2548, -0.0302, -0.0129],
         [ 0.7490, -0.3286, -0.1462,  ...,  0.0615, -0.0844,  0.3857],
         [ 0.4196,  0.3504, -0.1835,  ..., -0.2092,  0.6564,  0.0437],
         ...,
         [ 0.7356,  0.1710, -1.2289,  ..., -0.0389,  0.5717,  0.0603],
         [ 0.5751, -0.4045, -2.5969,  ..., -0.0122,  0.3050,  0.1939],
         [ 0.1517, -0.4045, -0.1892,  ..., -0.2258, -0.1274, -0.2186]]],
       grad_fn=<ViewBackward0>)

In [85]:
(initial_input == model(encoded, output_hidden_states=True).hidden_states[-1]).all()

tensor(True)

In [86]:
total_params = 0
for param in model.parameters():
  total_params += numel(param)                   # use numel to add up number of parameters

print(f"Number of params: {total_params:,}")

Number of params: 124,439,808


### Masked Attention
* Teaching GPT how to predict tokens while only having access to attention for past tokens.

## Hyperparameter Tuning
* Less Randomness, more natural sounding via Parameter Setting

In [87]:
import torch
import pandas as pd

In [88]:
phrase = "Just stay on the track and never look back!"
encoded_phrase = tokenizer(phrase, return_tensors='pt')
encoded_phrase

{'input_ids': tensor([[5703, 2652,  319,  262, 2610,  290, 1239,  804,  736,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [89]:
response = model(**encoded_phrase, output_attentions=True, output_hidden_states=True)

len(response.attentions), response.attentions[0].shape  #[0] is the first decoder

(12, torch.Size([1, 12, 10, 10]))

In [90]:
response.attentions[-1].shape   # [-1] is the final decoder
                                # [1,12,13,13] [batch_size,12head, 13x13 tokens]

torch.Size([1, 12, 10, 10])

In [91]:
encoded_phrase['input_ids'].shape

torch.Size([1, 10])

In [92]:
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0])
tokens

['Just',
 'Ġstay',
 'Ġon',
 'Ġthe',
 'Ġtrack',
 'Ġand',
 'Ġnever',
 'Ġlook',
 'Ġback',
 '!']

In [93]:
from flax.linen import attention
# Layer index 9, head 0.  Check out the almost 60% attention the token it is giving to the token class [layer][head][batch_size]
arr = response.attentions[9][0][0]

n_digits = 3

attention_df = pd.DataFrame((torch.round(arr * 10**n_digits) / (10**n_digits)).detach()).applymap(float)

attention_df.columns = tokens
attention_index = tokens

attention_df

,Just,Ġstay,Ġon,Ġthe,Ġtrack,Ġand,Ġnever,Ġlook,Ġback,!
0,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.989,0.011,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,0.983,0.009,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,0.902,0.017,0.028,0.053,0.000,0.000,0.000,0.000,0.000,0.000
4,0.852,0.009,0.012,0.025,0.102,0.000,0.000,0.000,0.000,0.000
5,0.905,0.013,0.007,0.019,0.050,0.006,0.000,0.000,0.000,0.000
6,0.818,0.016,0.015,0.037,0.094,0.011,0.010,0.000,0.000,0.000
7,0.949,0.003,0.003,0.018,0.013,0.004,0.007,0.003,0.000,0.000
8,0.919,0.008,0.009,0.015,0.013,0.004,0.004,0.007,0.023,0.000
9,0.422,0.013,0.011,0.030,0.008,0.004,0.002,0.001,0.002,0.508


In [94]:
!pip install bertviz
from bertviz import model_view

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [95]:
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0])  # rows of layers, columns of head
model_view(response.attentions, tokens)

<IPython.core.display.Javascript object>

In [96]:
response.hidden_states[-1].shape  # [batch, token, embedded_vector]

torch.Size([1, 10, 768])

In [97]:
response.logits.shape  # [1,13,50256]  [batch,token,logit]

######
# These logits are the outputs of the final language modeling layer which applies a feed forward layer to each of the 13 tokens
# Each of the 13 tokens is mapped to one of the 50,258 possible tokens in GPT-2.
#
# Each word has the next best word

torch.Size([1, 10, 50257])

In [98]:
pd.DataFrame(
    zip(tokens, tokenizer.convert_ids_to_tokens(response.logits.argmax(2)[0])),
    columns=['Sequence up until', 'Next token with highest probability']
)

,Sequence up until,Next token with highest probability
0,Just,Ġthe
1,Ġstay,Ġtuned
2,Ġon,Ġtop
3,Ġthe,Ġright
4,Ġtrack,.
5,Ġand,Ġkeep
6,Ġnever,Ġlet
7,Ġlook,Ġback
8,Ġback,.
9,!,Ċ


In [99]:
generator(phrase, max_length=20, num_return_sequences=1, do_sample=False)  # greedy search

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Just stay on the track and never look back!\n\nThe following is a list of all the'}]

In [100]:
generator(phrase, max_length=20, num_return_sequences=3, do_sample=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Just stay on the track and never look back! (Or just get on board...)\n\nThere'},
 {'generated_text': 'Just stay on the track and never look back!\n\nIn order to be part of your favorite'},
 {'generated_text': 'Just stay on the track and never look back!\n\nWe invite you to join us for the'}]

## BIAS in GPT
* People were noticing biases in those word2vec embeddings. This bias is true to this day and in some ways is exacerbated. if we are not careful, those biases will be transferred over into our downstream task

In [101]:
from transformers import pipeline, set_seed
from torch import tensor

generator = pipeline('text-generation', model='gpt2', tokenizer=tokenizer)
set_seed(0)

In [102]:
# Bias
generator("The holocaust was", max_length=10, num_return_sequences=10, temperature=0.8, num_beams=2)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The holocaust was a lie. It was a'},
 {'generated_text': 'The holocaust was a lie. It was a'},
 {'generated_text': 'The holocaust was an act of genocide. It'},
 {'generated_text': 'The holocaust was a complete fabrication. It was'},
 {'generated_text': 'The holocaust was not just a crime against humanity'},
 {'generated_text': 'The holocaust was a very real event, and'},
 {'generated_text': 'The holocaust was a genocide. It was a'},
 {'generated_text': 'The holocaust was a terrible and terrible crime,'},
 {'generated_text': 'The holocaust was a real thing.\n\n'},
 {'generated_text': 'The holocaust was a major event in the history'}]

In [103]:
generator("The earth is", max_length=10, num_return_sequences=10, temperature=0.8, num_beams=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The earth is not a flat place. It is'},
 {'generated_text': 'The earth is flat, and the sky is flat'},
 {'generated_text': 'The earth is a vast and beautiful place, and'},
 {'generated_text': 'The earth is full of life, and we are'},
 {'generated_text': 'The earth is a beautiful place, but it is'},
 {'generated_text': 'The earth is flat. The sky is flat.'},
 {'generated_text': 'The earth is full of life, and the stars'},
 {'generated_text': 'The earth is a beautiful place. It is a'},
 {'generated_text': 'The earth is a place of great beauty, and'},
 {'generated_text': 'The earth is a good place to live," he'}]

## Few-Shot Learning
* Zero-shot learning is when we would give a model a task description, but no examples prior.  The prompt is what's used to tell the model what to do.
* Sentiment analysis is not what GPT was trained to do explicitly, but implicitly it can figure out what sentiment analysis is through a few examples.

#### https://www.repustate.com/blog/social-listening-process/

In [104]:
print(generator("""Sentiment Analysis
Text: I hate it when my phone battery dies.
Sentiment: Negative
###
Text: My day has been really great!
Sentiment: Positive
###
Text: Not a fan when it is cloudy
Sentiment:""", top_k=2, temperature=0.1, max_length=55)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sentiment Analysis
Text: I hate it when my phone battery dies.
Sentiment: Negative
###
Text: My day has been really great!
Sentiment: Positive
###
Text: Not a fan when it is cloudy
Sentiment: Negative



In [105]:
print(generator("""Sentiment Analysis
Text: I feel terrible when facing unbearable bugs.
Sentiment: Negative
###
Text: My code works like a charm!
Sentiment: Positive
###
Text: I gave up on my code after many hours debugging.
Sentiment:""", top_k=2, temperature=0.1, max_length=55)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 56, but `max_length` is set to 55. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Sentiment Analysis
Text: I feel terrible when facing unbearable bugs.
Sentiment: Negative
###
Text: My code works like a charm!
Sentiment: Positive
###
Text: I gave up on my code after many hours debugging.
Sentiment: Negative


#### Q/A

In [106]:
print(generator("""Question/Answering
C: Google was founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University in California. Together they own about 14 percent of its shares and control 56 percent of the stockholder voting power through supervoting stock.
Q: When was Google founded?
A: 1998
###
C: Hugging Face is a company which develops social AI-run chatbot applications. It was established in 2016 by Clement Delangue and Julien Chaumond. The company is based in Brooklyn, New York, United States.
Q: What does Hugging Face develop?
A: social AI-run chatbot applications
###
C: The New York Jets are a professional American football team based in the New York metropolitan area. The Jets compete in the National Football League (NFL) as a member club of the league's American Football Conference (AFC) East division.
Q: What division do the Jets play in?
A:""", top_k=2, num_beams=2, max_length=215, temperature=0.5)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question/Answering
C: Google was founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University in California. Together they own about 14 percent of its shares and control 56 percent of the stockholder voting power through supervoting stock.
Q: When was Google founded?
A: 1998
###
C: Hugging Face is a company which develops social AI-run chatbot applications. It was established in 2016 by Clement Delangue and Julien Chaumond. The company is based in Brooklyn, New York, United States.
Q: What does Hugging Face develop?
A: social AI-run chatbot applications
###
C: The New York Jets are a professional American football team based in the New York metropolitan area. The Jets compete in the National Football League (NFL) as a member club of the league's American Football Conference (AFC) East division.
Q: What division do the Jets play in?
A: The AFC East
###
C:


#### Context-based Answering

In [107]:
# Same question as before, with no previous examples ie Zero-shot learning. Still works
print(generator(
    '''Question/Answering
C: The New York Jets are a professional American football team based in the New York metropolitan area. The Jets compete in the National Football League (NFL) as a member club of the league's American Football Conference (AFC) East division.
Q: What division do the Jets play in?
A:''',
    top_k=2, num_beams=2, max_length=80, temperature=0.5)[0]['generated_text']
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question/Answering
C: The New York Jets are a professional American football team based in the New York metropolitan area. The Jets compete in the National Football League (NFL) as a member club of the league's American Football Conference (AFC) East division.
Q: What division do the Jets play in?
A: The Jets play in the AFC East, which is the


#### Abstractive Summarization

#### Ref: https://huggingface.co/docs/transformers/model_doc/gpt2

In [108]:
to_summarize = """This training will focus on how the GPT family of models are used for NLP tasks including abstractive text summarization and natural language generation. The training will begin with an introduction to necessary concepts including masked self attention, language models, and transformers and then build on those concepts to introduce the GPT architecture. We will then move into how GPT is used for multiple natural language processing tasks with hands-on examples of using pre-trained GPT-2 models as well as fine-tuning these models on custom corpora.

GPT models are some of the most relevant NLP architectures today and it is closely related to other important NLP deep learning models like BERT. Both of these models are derived from the newly invented transformer architecture and represent an inflection point in how machines process language and context.

The Natural Language Processing with Next-Generation Transformer Architectures series of online trainings provides a comprehensive overview of state-of-the-art natural language processing (NLP) models including GPT and BERT which are derived from the modern attention-driven transformer architecture and the applications these models are used to solve today. All of the trainings in the series blend theory and application through the combination of visual mathematical explanations, straightforward applicable Python examples within hands-on Jupyter notebook demos, and comprehensive case studies featuring modern problems solvable by NLP models. (Note that at any given time, only a subset of these classes will be scheduled and open for registration.)"""

In [109]:
print(generator(
    f"""Summarization Task:\n{to_summarize}\nTL;DR:""", 
      max_length=512,
      top_k=3,
      num_beams=3,
      temperature=0.8,
      num_return_sequences = 2,
      no_repeat_ngram_size = 3,
      early_stopping = True
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Summarization Task:
This training will focus on how the GPT family of models are used for NLP tasks including abstractive text summarization and natural language generation. The training will begin with an introduction to necessary concepts including masked self attention, language models, and transformers and then build on those concepts to introduce the GPT architecture. We will then move into how GPT is used for multiple natural language processing tasks with hands-on examples of using pre-trained GPT-2 models as well as fine-tuning these models on custom corpora.

GPT models are some of the most relevant NLP architectures today and it is closely related to other important NLP deep learning models like BERT. Both of these models are derived from the newly invented transformer architecture and represent an inflection point in how machines process language and context.

The Natural Language Processing with Next-Generation Transformer Architectures series of online trainings provides a

In [110]:
print(generator(
    f"""Summarization Task:\n{to_summarize}\nTL;DR:""", 
      max_length=512,
      top_k=3,
      num_beams=2,
      temperature=0.8,
      num_return_sequences = 2,
      # no_repeat_ngram_size = 3,
      early_stopping = True
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Summarization Task:
This training will focus on how the GPT family of models are used for NLP tasks including abstractive text summarization and natural language generation. The training will begin with an introduction to necessary concepts including masked self attention, language models, and transformers and then build on those concepts to introduce the GPT architecture. We will then move into how GPT is used for multiple natural language processing tasks with hands-on examples of using pre-trained GPT-2 models as well as fine-tuning these models on custom corpora.

GPT models are some of the most relevant NLP architectures today and it is closely related to other important NLP deep learning models like BERT. Both of these models are derived from the newly invented transformer architecture and represent an inflection point in how machines process language and context.

The Natural Language Processing with Next-Generation Transformer Architectures series of online trainings provides a

## Prompt Engineering

#### Ref: https://newskills.ai/playground/

### Task1 Language Translation

In [ ]:
# "Silence is golden."                       STEP1: ask ChatGPT to translate to Japanese
# "沈黙は金"
# im lặng là vàng                            STEP2: validate the translation with Google Translate to another language Vietnamese

Write a short and creative story about a cat named Whiskers.

Try to be as creative as possible.

Limit the length of your response to 50 words.

### Task2: Write frienly email to a client

Write a friendly email to a client, John Smith, thanking him for his recent purchase of our premium software package and offering assistance if he has any questions.

* Thank you for your purchase

### Task3: Write an Instagram caption

Craft a playful and tempting Instagram caption to promote our bakery's new chocolate chip cookie ice cream sandwich, available for a limited time.

Indulge in the ultimate dessert experience with our new chocolate chip cookie ice cream sandwich! 😍🍪🍦 Trust us, your taste buds won't be able to resist this limited-time treat. Hurry in and grab yours before they're gone! 🏃‍♀️🏃‍♂️ #CookieLoversUnite #IceCreamDreams #LimitedTimeOnly #SweetTreats #BakeryLife 🍭🍰🍩

### Task4: Write a comical Haiku

Write a comical haiku describing a heavy storm in a winter day.

Flakes falling so fast, Wind makes snowflakes fly at last, Cold feet, time to blast!

Write a comical haiku, imitating Matsuo Basho, describing a heavy storm in a winter day.
* Winter storm so strong, bellows like a raging bull, I hide under wool.

Write a comical haiku, imitating Natume Sooseki, describing a heavy storm in a winter day.
* Wind howls and snow falls My umbrella flies away Winter hates my guts.

### Task5: Step-by-Step instruction to build a rocket

Show me a short but precise step-by-step instructions to build a rocket.

* Building a rocket requires precision and careful attention to detail. Here are some basic, high-level steps to get started: 1. Designing: Before starting to build the rocket, it's important to design it well. Decide on the design, size and structure which suits your requirements. 2. Assemble Materials: Gather all the materials needed for the rocket. This may include PVC piping, fins, motor casing, launch pad, rocket engine, and recovery system. 3. Cut and Shape the Body: Cut the PVC piping to the desired length and shape down any unnecessary components. 4. Attach the Fins: Fit and attach the fins to the body of the rocket. Ensure they are securely fastened for a safe launch. 5. Install the Motor: Install the rocket motor into its casing, ensuring that it is held in place snugly and can withstand the rocket's thrust. 6. Connect the Recovery System: Install the recovery system, which can be in the form of a parachute, streamer or any other recovery mechanism. 7. Test and Launch: Test the rocket's stability, with all components correctly in place. After making sure the components are all safe and secure to fly, launch the rocket from a safe location and enjoy your flight! Note: Rocket building can be a dangerous activity, so always prioritize safety and carefully follow all instructions from any textbooks or authoritative sources

### Task6: Give Life-advice based on Zodiac Sign

My birthday is today, give me some life-advice based on my Zodiac sign.
* Happy birthday! What's your Zodiac sign? I can provide you with some life-advice based on your Zodiac sign traits.

### Task7: Give Best Exercise To Increase Muscle Mass

I am a very slim person. What's best exercise to increase muscle mass?
* Strength training exercises such as weight lifting or bodyweight exercises like push-ups, pull-ups, and squats are excellent for increasing muscle mass. To facilitate muscle growth, you need to create muscle tension and damage the muscle fibers, which strength training exercises do through repetitive movements and use of progressive resistance. However, keep in mind that building muscle also requires proper nutrition such as a diet high in protein, adequate hydration, and enough rest and recovery time between workouts. It's also essential to consult with a healthcare provider or a certified fitness professional to tailor a workout program that suits your fitness goals, preferences, and overall health status.

### Task8: Show a 20-minutes Muscle Building Routine

Show me a 20-minutes muscle building routine.
* Sure! Here's a 20-minute muscle building routine that targets your entire body: 1. Warm-up (2 minutes): Start with a few minutes of cardio such as jumping jacks, running in place, or high-knees. 2. Squats (2 minutes): Stand with your feet shoulder-width apart, squat down until your thighs are parallel to the ground. Squeeze your glutes as you rise back up. Repeat for 2 minutes. 3. Push-ups (2 minutes): Get into a plank position with your hands shoulder-width apart, perform a push-up by lowering your chest to the ground and pushing back up. Repeat for 2 minutes. 4. Jumping lunges (2 minutes): Start in a lunge position, jump up and switch legs midair. Land softly with your opposite foot forward into a lunge. Repeat for 2 minutes. 5. Dumbbell curls (2 minutes): Stand with your feet shoulder-width apart, holding dumbbells by your sides. Bend your elbows and lift the weights to your shoulders while keeping your arms close to your body. Slowly lower the dumbbells back down. Repeat for 2 minutes. 6. Plank variations (4 minutes): Mix in various plank variations such as side planks, plank jacks, and up-down planks for 4 minutes. 7. Cool-down (2 minutes): Finish with a few stretches such as quad stretches, hamstring stretches, and chest stretches to help your muscles recover. Remember, you can increase or decrease the intensity of this workout depending on your fitness level and endurance. It's also essential to maintain proper form throughout the workout to prevent injury.

### Task9: As an ML engineer, write an AI model to detect fraud.

As a machine-learning engineer, write a AI model to detect fraud using PyTorch.

Sure, here's an AI model to detect fraud using PyTorch: ``` import torch import torch.nn as nn class FraudDetector(nn.Module): def __init__(self, input_size, hidden_size, num_classes): super(FraudDetector, self).__init__() self.hidden = nn.Linear(input_size, hidden_size) self.dropout = nn.Dropout(p=0.2) self.relu = nn.ReLU() self.output = nn.Linear(hidden_size, num_classes) def forward(self, x): h = self.hidden(x) h = self.relu(h) h = self.dropout(h) out = self.output(h) return out ``` This is a simple neural network that takes in input data and passes it through a linear layer, a ReLU activation function, and a dropout layer to prevent overfitting. The output is passed through another linear layer to get the final output. To train this model, you can use a combination of supervised learning and anomaly detection. You can label certain data as fraudulent and use it to train the model using standard techniques such as backpropagation. You can also use unsupervised methods such as clustering to identify unusual patterns in the data that might indicate fraud. Once the model is trained and optimized, you can use it to classify new data as either fraudulent or not fraudulent. This can help automate fraud detection and improve the efficiency and accuracy of fraud detection systems.

### Task10: Give Life-Coach advice

As my life-coach, tell me some best practices to stay positive in a difficult situation.

Certainly! Here are some best practices to stay positive in a difficult situation: 1. Practice gratitude: In challenging times, it can be helpful to take a few moments to think about what you are grateful for. Even in the worst of circumstances, there are usually things to be thankful for, such as your health, your loved ones, or the fact that you have a roof over your head. 2. Focus on what you can control: In many situations, there are things that are beyond our control. Instead of getting bogged down by them, focus on the things you can control. For example, you may not be able to control the weather or the traffic, but you can control your attitude towards them. 3. Stay connected: When things get tough, it's important to stay connected with friends, family, and loved ones. Talking to others can help provide perspective and support. 4. Take care of yourself: When you're feeling stressed or overwhelmed, it can be easy to neglect self-care. However, taking care of yourself is essential for maintaining a positive attitude. Get enough sleep, eat well, and make time for activities that you enjoy. 5. Practice mindfulness: Mindfulness is the practice of being present in the moment, without judgment. This can be a helpful tool for managing stress and maintaining a positive outlook. Try to focus on the present moment, rather than worrying about the past or future. Remember, staying positive in difficult situations is not always easy, but with practice and patience, it's possible to develop a more optimistic outlook.

### Task11: Help with writing essay

I need to write an essay about Climate Change and its effect on crop-generation. Can you help?

* Of course! Here's a potential outline you could use for your essay: I. Introduction - Climate change and crop-generation - The importance of this topic II. The impact of climate change on crop-generation - Higher temperatures and more extreme weather patterns - Changes in precipitation - The effects of all of these changes on crop yield and quality III. Specific effects of climate change on crops - How different crops are affected, depending on the region - Drought and water stress - Heat stress - Changes in pests and diseases IV. Mitigating the effects of climate change on crop-generation - Sustainable farming practices - Policies and regulations - Public awareness and education V. Conclusion - The importance of addressing climate change and its effect on crop-generation - Potential solutions and ways to move forward Remember to use credible sources for your research and to tailor your writing to your specific audience. Good luck with your essay!

### Task12: Research on environmental impact of training a large GPT model

What's the environmental impact of training a GPT model?

Great info. Give me specific carbon emissions when training a GPT model.

Explain the environmental impact of training a GPT. Next, give specific carbon-emissions amount when training a GPT model.

List the top 5 cause of obesity then write a catchy tune to encourage young men to sign up for gym membership.

## Summary and Conclusion
Natural Language Generation (NLG) with GPT (Generative Pre-trained Transformer):
•	This project focuses on exploring the application side of GPT (Generative Pre-trained Transformer) and its capabilities as a text generator. We delve into the inner workings of GPT, including masked attention and hyperparameter tuning, to improve the quality of generated text. Additionally, we discuss the issue of bias in GPT and the importance of being aware of it when using GPT for downstream tasks.

•	In this project, we also explore the concept of few-shot learning, where GPT can learn to perform new tasks with just a few examples. Our findings suggest that GPT is a powerful tool for natural language generation and has applications in various industries, including marketing, customer service, and content creation.

•	Prompt engineering is another powerful tool that we explore in this project to further enhance the capabilities of GPT. By providing specific prompts to GPT, we can control the output and generate text that is relevant to a specific topic or task. For example, a reader can just type in questions and see the responses generated by GPT. Prompt engineering has the potential to revolutionize the way we use natural language generation in various industries.

•	Finally, this project provides insights into how GPT can be used to improve text generation and generate high-quality content.



## References

#### https://newskills.ai/playground/
#### https://huggingface.co/docs/transformers/model_doc/gpt2
#### https://www.repustate.com/blog/social-listening-process/